In [ ]:
# -*- coding: utf-8 -*-
import cv2
import numpy as np

# ==============================
# 1) 读取图像
# ==============================
img = cv2.imread("../image/road.png")
if img is None:
    raise FileNotFoundError("未找到图片")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 150)

# ==============================
# 2) 霍夫直线检测
# ==============================
# cv2.HoughLines(image, rho, theta, threshold)
#   - image     : Canny 边缘图（必须是二值边缘图）
#   - rho       : 距离分辨率（像素）
#   - theta     : 角度分辨率（弧度）
#   - threshold : 累计器阈值（越大→越少直线）
lines = cv2.HoughLines(edges, 1, np.pi / 180, 100)
if lines is not None:
    for line in lines:
        rho, theta = line[0]
        a, b = np.cos(theta), np.sin(theta)
        x0, y0 = a * rho, b * rho
        x1, y1 = int(x0 + 1000 * (-b)), int(y0 + 1000 * (a))
        x2, y2 = int(x0 - 1000 * (-b)), int(y0 - 1000 * (a))
        cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)

# ==============================
# 3) 霍夫圆检测
# ==============================
# cv2.HoughCircles(image, method, dp, minDist[, param1[, param2[, minRadius[, maxRadius]]]])
#   - image     : 输入灰度图
#   - method    : 一般用 cv2.HOUGH_GRADIENT
#   - dp        : 累计器分辨率比例（1=相同大小）
#   - minDist   : 圆心之间最小距离
#   - param1    : Canny 高阈值
#   - param2    : 累计阈值（越小检测越多伪圆）
#   - minRadius/maxRadius : 圆半径范围
circles = cv2.HoughCircles(
    gray,
    cv2.HOUGH_GRADIENT,
    dp=1,
    minDist=50,
    param1=100,
    param2=30,
    minRadius=10,
    maxRadius=100,
)
if circles is not None:
    circles = np.uint16(np.around(circles))
    for c in circles[0, :]:
        cv2.circle(img, (c[0], c[1]), c[2], (0, 255, 0), 2)

# ==============================
# 4) 显示结果
# ==============================
cv2.imshow("Edges", edges)
cv2.imshow("Hough Result", img)
cv2.waitKey(0)
cv2.destroyAllWindows()